In [1]:
#import gmsh # used in pygmsh
import pygmsh
#import meshio # used in pygmsh
import numpy as np

use_cpml = True

# Initialize empty geometry using the build in kernel in GMSH
with pygmsh.geo.Geometry() as model:

    # define global parameters
    H = 20 * 1000  # domain height in meter
    L = 100 * 1000  # domain width in meter

    H_w = 4 * 1000  # water depth in meter
    H_g = (H - H_w)  # subsurface depth in meter

    # element size for both water and subsurface domain
    #lc_w = 0.09375 * 1000
    #lc_g = 0.075 * 1000

    # test size
    lc_w = 0.5 * 1000
    lc_g = 0.375 * 1000

    if use_cpml:
        # pml layer thickness
        pml = lc_w * 3

    top_pml = False


    """ node ids without pml layer
    5             6

    4             3 <--- subsurface topo


    1             2

    """

    """ node ids with pml layer
    16  15            14  13 <--- top_pml = True

    17   O            O   12
                w
    18   O            O   11

                g

    19   O            O   10

    20   7            8   9
    """


    # points
    p1 = model.add_point((0, 0, 0), lc_g)
    p2 = model.add_point((L, 0, 0), lc_g)
    p3 = model.add_point((L, H_g, 0), lc_g)
    p4 = model.add_point((0, H_g, 0), lc_g)
    p5 = model.add_point((0, H, 0), lc_w)
    p6 = model.add_point((L, H, 0), lc_w)

    # points for pml layer outside the domain
    if use_cpml:
        p7 = model.add_point((0, -pml, 0), lc_g)
        p8 = model.add_point((L, -pml, 0), lc_g)
        p9 = model.add_point((L + pml, -pml, 0), lc_g)
        p10 = model.add_point((L + pml, 0, 0), lc_g)
        p11 = model.add_point((L + pml, H_g, 0), lc_g)
        p12 = model.add_point((L + pml, H, 0), lc_w)

        if top_pml:
            p13 = model.add_point((L + pml, H + pml, 0), lc_w)
            p14 = model.add_point((L, H + pml, 0), lc_w)
            p15 = model.add_point((0, H + pml, 0), lc_w)
            p16 = model.add_point((-pml, H + pml, 0), lc_w)

        p17 = model.add_point((-pml, H, 0), lc_w)
        p18 = model.add_point((-pml, H_g, 0), lc_g)
        p19 = model.add_point((-pml, 0, 0), lc_g)
        p20 = model.add_point((-pml, -pml, 0), lc_g)

    # lines
    l_bot = model.add_line(p1, p2)
    l_right_g = model.add_line(p2, p3)
    l_bound_wg = model.add_line(p3, p4)
    l_left_g = model.add_line(p4, p1)
    l_top = model.add_line(p5, p6)
    l_right_w = model.add_line(p3, p6)
    l_left_w = model.add_line(p4, p5)

    # line loop
    ll_g = model.add_curve_loop([l_bot, l_right_g, l_bound_wg, l_left_g])
    ll_w = model.add_curve_loop([-l_bound_wg, l_right_w, -l_top, -l_left_w])

    # plane surface
    ps_g = model.add_plane_surface(ll_g)
    ps_w = model.add_plane_surface(ll_w)

    #
    # PML layer
    #
    if use_cpml:
        # lines
        l_bot_pml = model.add_line(p7, p8)
        l_br_h_pml = model.add_line(p8, p9)
        l_br_v_pml = model.add_line(p9, p10)
        l_br_h2_pml = model.add_line(p2, p10)
        l_br_v2_pml = model.add_line(p2, p8)
        l_r_g_pml = model.add_line(p10, p11)
        l_r_gw_pml = model.add_line(p3, p11)
        l_r_w_pml = model.add_line(p11, p12)

        if top_pml:
            l_tr_v_pml = model.add_line(p12, p13)
            l_tr_h_pml = model.add_line(p13, p14)
            l_top_pml = model.add_line(p14, p15)
            l_tl_h_pml = model.add_line(p15, p16)
            l_tl_v_pml = model.add_line(p16, p17)
            l_tr_v2_pml = model.add_line(p6, p14)
            l_tl_v2_pml = model.add_line(p5, p15)

        l_tr_h2_pml = model.add_line(p6, p12)
        l_tl_h2_pml = model.add_line(p5, p17)

        l_l_w_pml = model.add_line(p17, p18)
        l_l_gw_pml = model.add_line(p4, p18)
        l_l_g_pml = model.add_line(p18, p19)
        l_bl_v_pml = model.add_line(p19, p20)
        l_bl_h_pml = model.add_line(p20, p7)
        l_bl_v2_pml = model.add_line(p1, p7)
        l_bl_h2_pml = model.add_line(p1, p19)

        # line loop
        ll_bot_pml = model.add_curve_loop([l_bot_pml, -l_br_v2_pml, -l_bot, l_bl_v2_pml])
        ll_br_pml = model.add_curve_loop([l_br_h_pml, l_br_v_pml, -l_br_h2_pml, l_br_v2_pml])
        ll_r_g_pml = model.add_curve_loop([l_br_h2_pml, l_r_g_pml, -l_r_gw_pml, -l_right_g])
        ll_r_w_pml = model.add_curve_loop([l_r_gw_pml, l_r_w_pml, -l_tr_h2_pml, -l_right_w])

        if top_pml:
            ll_tr_pml = model.add_curve_loop([l_tr_h2_pml, l_tr_v_pml, l_tr_h_pml, -l_tr_v2_pml])
            ll_top_pml = model.add_curve_loop([l_tr_v2_pml, l_top_pml, -l_tl_v2_pml, l_top])
            ll_tl_pml = model.add_curve_loop([l_tl_v2_pml, l_tl_h_pml, l_tl_v_pml, -l_tl_h2_pml])

        ll_l_w_pml = model.add_curve_loop([l_tl_h2_pml, l_l_w_pml, -l_l_gw_pml, l_left_w])
        ll_l_g_pml = model.add_curve_loop([l_l_gw_pml, l_l_g_pml, -l_bl_h2_pml, -l_left_g])
        ll_bl_pml = model.add_curve_loop([l_bl_h2_pml, l_bl_v_pml, l_bl_h_pml, -l_bl_v2_pml])

        # plane surface
        ps_bot_pml = model.add_plane_surface(ll_bot_pml)
        ps_br_pml = model.add_plane_surface(ll_br_pml)
        ps_r_g_pml = model.add_plane_surface(ll_r_g_pml)
        ps_r_w_pml = model.add_plane_surface(ll_r_w_pml)

        if top_pml:
            ps_tr_pml = model.add_plane_surface(ll_tr_pml)
            ps_top_pml = model.add_plane_surface(ll_top_pml)
            ps_tl_pml = model.add_plane_surface(ll_tl_pml)

        ps_l_w_pml = model.add_plane_surface(ll_l_w_pml)
        ps_l_g_pml = model.add_plane_surface(ll_l_g_pml)
        ps_bl_pml = model.add_plane_surface(ll_bl_pml)


    # recombine surfaces
    if use_cpml:
        if top_pml:
            model.set_recombined_surfaces([ps_g, ps_w,
                                           ps_bot_pml, ps_br_pml, ps_r_g_pml, ps_r_w_pml,
                                           ps_tr_pml, ps_top_pml, ps_tl_pml, ps_l_w_pml,
                                           ps_l_g_pml, ps_bl_pml])
        else:
            model.set_recombined_surfaces([ps_g, ps_w,
                                           ps_bot_pml, ps_br_pml, ps_r_g_pml, ps_r_w_pml,
                                           ps_l_w_pml, ps_l_g_pml, ps_bl_pml])

    else:
        model.set_recombined_surfaces([ps_g, ps_w])

    # synchronize
    model.synchronize()

    # physical groups
    # boundary
    if not use_cpml:
        pg_top = model.add_physical(l_top, label="Top")
        pg_bot = model.add_physical(l_bot, label="Bottom")
        pg_left = model.add_physical([l_left_g, l_left_w], label="Left")
        pg_right = model.add_physical([l_right_g, l_right_w], label="Right")
    else:
        if top_pml:
            pg_top = model.add_physical([l_top_pml, l_tr_h_pml, l_tl_h_pml], label="Top")
            pg_left = model.add_physical([l_tl_v_pml, l_l_w_pml, l_l_g_pml, l_bl_v_pml], label="Left")
            pg_right = model.add_physical([l_br_v_pml, l_r_g_pml, l_r_w_pml, l_tr_v_pml], label="Right")
        else:
            pg_top = model.add_physical([l_top, l_tr_h2_pml, l_tl_h2_pml], label="Top")
            pg_left = model.add_physical([l_l_w_pml, l_l_g_pml, l_bl_v_pml], label="Left")
            pg_right = model.add_physical([l_br_v_pml, l_r_g_pml, l_r_w_pml], label="Right")

        pg_bot = model.add_physical([l_bot_pml, l_br_h_pml, l_bl_h_pml], label="Bottom")
    # material
    if not use_cpml:
        pg_g = model.add_physical(ps_g, label="M1")  # subsurface
        pg_w = model.add_physical(ps_w, label="M2")  # water
    else:

        pg_g = model.add_physical([ps_g, ps_bot_pml, ps_bl_pml, ps_br_pml, ps_l_g_pml, ps_r_g_pml], label="M1")

        if top_pml:
            pg_w = model.add_physical([ps_w, ps_top_pml, ps_tl_pml, ps_tr_pml, ps_l_w_pml, ps_r_w_pml], label="M2")
        else:
            pg_w = model.add_physical([ps_w, ps_l_w_pml, ps_r_w_pml], label="M2")

        pg_pml_x = model.add_physical([ps_r_g_pml, ps_r_w_pml, ps_l_g_pml, ps_l_w_pml], label="PML_X") # PML in x direction
        if top_pml:
            pg_pml_y = model.add_physical([ps_bot_pml, ps_top_pml], label="PML_Y")
            pg_pml_xy = model.add_physical([ps_br_pml, ps_tl_pml, ps_tr_pml, ps_bl_pml], label="PML_XY")
        else:
            pg_pml_y = model.add_physical([ps_bot_pml], label="PML_Y")
            pg_pml_xy = model.add_physical([ps_br_pml, ps_bl_pml], label="PML_XY")

    # Generate mesh (meshio object)
    mesh = model.generate_mesh(dim=2, order=2, verbose=True)

mesh.write("mesh.xdmf",
           file_format='xdmf')

#mesh.write("mesh.msh",
#           file_format='gmsh22')

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 12 (Line)
Info    : [ 50%] Meshing curve 13 (Line)
Info    : [ 60%] Meshing curve 14 (Line)
Info    : [ 60%] Meshing curve 15 (Line)
Info    : [ 70%] Meshing curve 16 (Line)
Info    : [ 70%] Meshing curve 17 (Line)
Info    : [ 80%] Meshing curve 18 (Line)
Info    : [ 80%] Meshing curve 19 (Line)
Info    : [ 80%] Meshing curve 20 (Line)
Info    : [ 90%] Meshing curve 21 (Line)
Info    : [ 90%] Meshing curve 22 (Line)
Info    : [100%] Meshing curve 23 (Line)
Info    : [100%] Meshing curve 24 (Line)
I

Info    : [  0%] Blossom: 40035 internal 624 closed
Info    : [  0%] Blossom recombination completed (Wall 4.00456s, CPU 3.94816s): 13384 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.814692, min Q = 0.485156
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 20%] Blossom: 7478 internal 488 closed
Info    : [ 20%] Blossom recombination completed (Wall 0.201732s, CPU 0.202961s): 2536 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.794224, min Q = 0.413601
Info    : [ 30%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 30%] Blossom: 3898 internal 544 closed
Info    : [ 30%] Blossom recombination completed (Wall 0.0859613s, CPU 0.087345s): 1368 quads, 0 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.809861, min Q = 0.478976
Info    : [ 40%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 40%] Blossom: 55 internal 16 closed
Info    : [ 40%] Blossom recombination completed (Wall 0.00871975s, CPU 

In [2]:
mesh

<meshio mesh object>
  Number of points: 72093
  Number of cells:
    line3: 1268
    quad9: 17873
    vertex: 16
  Cell sets: Top, Left, Right, Bottom, M1, M2, PML_X, PML_Y, PML_XY

In [4]:
from meshio2spec2d import *

mio2spec = Meshio2Specfem2D(mesh)
mio2spec.write("test")

Time elapsed for nodes:  0.26607608795166016  seconds
Time elapsed for mesh:  0.10729503631591797  seconds
material keys:  ['M1', 'M2']
cell_id_offset:  599
Time elapsed for material:  0.0733652114868164  seconds
Time elapsed for surf:  7.4610443115234375  seconds
Time elapsed for cpml:  1.2932682037353516  seconds
Time elapsed:  9.20169448852539  seconds


In [7]:
# write MESH/nummaterial_velocity_file separately

"""
# format:
#(1)domain_id #(2)material_id #(3)rho #(4)vp #(5)vs #(6)Q_k #(7)Q_mu #(8)ani
#
#  where
#     domain_id          : 1=acoustic / 2=elastic / 3=poroelastic
#     material_id        : POSITIVE integer identifier of material block
#     rho                : density
#     vp                 : P-velocity
#     vs                 : S-velocity
#     Q_k                : 9999 = no Q_kappa attenuation
#     Q_mu               : 9999 = no Q_mu attenuation
#     ani                : 0=no anisotropy/ 1,2,.. check with aniso_model.f90
#
# example:
# 2   1 2300 2800 1500 9999.0 9999.0 0
#
# or
#
#(1)domain_id #(2)material_id  tomography elastic  #(3)filename #(4)positive
#
#  where
#     domain_id : 1=acoustic / 2=elastic / 3=poroelastic
#     material_id        : NEGATIVE integer identifier of material block
#     filename           : filename of the tomography file
#     positive           : a positive unique identifier
#
# example:
# 2  -1 tomography elastic tomo.xyz 1
"""

fname = "MESH/nummaterial_velocity_file"

# M1: subsurface
M1_params = "2 1 1850.d0 2400.d0 1200.d0 0 0 9999 9999 0 0 0 0 0 0"

# M2: water
M2_params = "1 1 1020.d0 1500.d0 0 0 0 9999 9999 0 0 0 0 0 0"

with open(fname, "w") as f:
    f.write(M1_params + "\n")
    f.write(M2_params + "\n")